In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import random
import pandas as pd

from PIL import Image
from scipy import ndimage

from glob import glob
from tqdm import tqdm
import shutil

tfk = tf.keras
tfkl = tfk.layers

In [ ]:
DB_BASE = '''dataset folder location'''
TRAIN_DIR_BASE = DB_BASE + '/training'

IMAGE_SIZE = (256,256)
CHANNELS = 3
INPUT_SHAPE = (IMAGE_SIZE)+(CHANNELS,)

BATCH_SIZE = 64
EPOCHS = 500

labels = ['Apple',        # 0
          'Blueberry',    # 1
          'Cherry',       # 2
          'Corn',         # 3
          'Grape',        # 4
          'Orange',       # 5
          'Peach',        # 6
          'Pepper',       # 7
          'Potato',       # 8
          'Raspberry',    # 9
          'Soybean',      # 10
          'Squash',       # 11
          'Strawberry',   # 12
          'Tomato'        # 13
]

In [ ]:
# vertical flips
def v_flip(image):
    return np.flip(image, axis=0)

# horizontal flip
def h_flip(image):
    return np.flip(image, axis=1)

# vertical shift
def v_shift(image, shift):
    return np.roll(image, int(shift*256), axis=0)

# horizontal shift
def h_shift(image, shift):
    return np.roll(image, int(shift*256), axis=1)

# rotation
def rotate(image, angle):
    image = Image.fromarray(image.astype('uint8'))
    image = image.rotate(angle)
    return np.array(image)

# crops central region of image
def crop(image, ratio):
    image = Image.fromarray(image.astype('uint8'))
    w, h = image.size

    ratio /= 2
    left = w * ratio
    right = w - left
    top = w * ratio
    bottom = h - top

    image = image.crop((left, top, right, bottom))
    image = image.resize((256,256))
    return np.array(image)

In [ ]:
def get_accuracy(model,augs = {}):

  # calculate actual model predictions
  datagen = ImageDataGenerator(validation_split=0.1,
                               preprocessing_function='''funzione per preprocessing usata nel training''')

  val_generator = datagen.flow_from_directory(
      TRAIN_DIR_BASE,
      target_size=(IMAGE_SIZE[0],IMAGE_SIZE[1]),
      color_mode = 'rgb',
      batch_size=BATCH_SIZE,
      class_mode='categorical',
      subset='validation',
      seed=seed,
      shuffle=False)
  
  prediction_probs = model.predict(val_generator)

  # calculate predictions for the augmentations
  def apply_augmentations(img):
    img = '''funzione per preprocessing usata nel training'''(img)
    
    for aug in augs.keys():
      if augs[aug] != False:
        if augs[aug] == True:
          img = aug(img)
        else:
          img = aug(img, augs[aug])

    return img

  datagen = ImageDataGenerator(validation_split=0.1,
                               preprocessing_function=apply_augmentations)

  val_generator = datagen.flow_from_directory(
      TRAIN_DIR_BASE,
      target_size=(IMAGE_SIZE[0],IMAGE_SIZE[1]),
      color_mode = 'rgb',
      batch_size=BATCH_SIZE,
      class_mode='categorical',
      subset='validation',
      seed=seed,
      shuffle=False)
  
  # sum to previous predictions
  prediction_probs = np.add(prediction_probs, model.predict(val_generator))

  # get actual predictions (higher predicted probability)
  predictions = []
  for el in prediction_probs:
    predictions.append(np.argmax(el,axis = 0))

  # create confusion matrix
  conf_mat = np.zeros((14,14))
  for (pr,label) in zip(predictions,val_generator.labels):
    conf_mat[label][pr] += 1

  # accuracy
  true_positives = np.trace(conf_mat)
  accuracy = float(true_positives) / conf_mat.sum()

  return accuracy

In [ ]:
# values to test

v_flip_values = [True, False]
h_flip_values = [True, False]
v_shift_values = [x for x in np.arange(0.25, 0.75, 0.25)]
h_shift_values = [x for x in np.arange(0.25, 0.75, 0.25)]
rotation_values = [x for x in range(60, 360, 60)]
crop_values = [x for x in np.arange(0.25, 1, 0.25)]

In [ ]:
# find optimal value for separate augmentations

model = model = tfk.models.load_model('model to test')
accuracies = []

# vertical flip
vf_accuracies = {}

top_acc = 0
best_vf_val = 0
for val in tqdm(v_flip_values):
  acc = get_accuracy(model,{v_flip: val})
  vf_accuracies[val] = acc
  if acc > top_acc:
    best_vf_val = val
  accuracies.append({'v_flip-' + ('True' if val else 'False'): acc})

# horizontal flip
hf_accuracies = {}

top_acc = 0
best_hf_val = 0
for val in tqdm(h_flip_values):
  acc = get_accuracy(model,{h_flip: val})
  hf_accuracies[val] = acc
  if acc > top_acc:
    best_hf_val = val
  accuracies.append({'h_flip-' + ('True' if val else 'False'): acc})

# vertical shift
vs_accuracies = {}

top_acc = 0
best_vs_val = 0
for val in tqdm(v_shift_values):
  acc = get_accuracy(model,{v_shift: val})
  vs_accuracies[val] = acc
  if acc > top_acc:
    best_vs_val = val
  accuracies.append({'v_shift-' + str(val): acc})

# horizontal shift
hs_accuracies = {}

top_acc = 0
best_hs_val = 0
for val in tqdm(h_shift_values):
  acc = get_accuracy(model,{h_shift: val})
  hs_accuracies[val] = acc
  if acc > top_acc:
    best_hs_val = val
  accuracies.append({'h_shift-' + str(val) : acc})

# rotation
rot_accuracies = {}

top_acc = 0
best_rot_val = 0
for val in tqdm(rotation_values):
  acc = get_accuracy(model,{rotate: val})
  rot_accuracies[val] = acc
  if acc > top_acc:
    best_rot_val = val
  accuracies.append({'rotate-' + str(val) : acc})

# crop
cr_accuracies = {}

top_acc = 0
best_cr_val = 0
for val in tqdm(crop_values):
  acc = get_accuracy(model,{crop: val})
  cr_accuracies[val] = acc
  if acc > top_acc:
    best_cr_val = val
  accuracies.append({'crop-' + str(val) : acc})

  0%|          | 0/2 [00:00<?, ?it/s]

Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.


 50%|█████     | 1/2 [06:38<06:38, 398.00s/it]

Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.


  0%|          | 0/2 [00:00<?, ?it/s]

...

 33%|███▎      | 1/3 [00:14<00:28, 14.41s/it]

Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.


 67%|██████▋   | 2/3 [00:28<00:14, 14.27s/it]

Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.


100%|██████████| 3/3 [00:42<00:00, 14.15s/it]


In [ ]:
accuracies

[{'v_flip-True': 0.9490373725934315},
 {'v_flip-False': 0.950736126840317},
 {'h_flip-True': 0.9535673839184597},
 {'h_flip-False': 0.950736126840317},
 {'v_shift-0.25': 0.9518686296715742},
 {'v_shift-0.5': 0.9513023782559457},
 {'h_shift-0.25': 0.9524348810872028},
 {'h_shift-0.5': 0.9524348810872028},
 {'rotate-60': 0.9416761041902605},
 {'rotate-120': 0.9331823329558324},
 {'rotate-180': 0.9479048697621744},
 {'rotate-240': 0.9371460928652322},
 {'rotate-300': 0.9320498301245753},
 {'crop-0.25': 0.9377123442808607},
 {'crop-0.5': 0.8924122310305775},
 {'crop-0.75': 0.7916194790486977}]

In [ ]:
# find optimal values when combining 2 and 3 augmentations
# augmentations are applied with their previously calculated best value except for flips

best_augs = {
    v_flip: True,
    h_flip: True,
    v_shift: best_vs_val,
    h_shift: best_hs_val,
    rotate: best_rot_val,
    crop: best_cr_val
}

aug_names = {
  v_flip: 'v_flip',
  h_flip: 'h_flip',
  v_shift: 'v_shift',
  h_shift: 'h_shift',
  rotate: 'rotate',
  crop: 'crop' 
}

# test combinations of two augmentations
for aug1 in best_augs.keys():
  for aug2 in best_augs.keys():
    if aug1 != aug2:
      x = {
          aug1: best_augs[aug1],
          aug2: best_augs[aug2]
      }
      acc = get_accuracy(model,x)
      print(aug_names[aug1],
            aug_names[aug2],
            '->',
            acc)
      s = aug_names[aug1] + '-' + aug_names[aug2]
      accuracies.append({s: acc})

# test combinations of three augmentations
for aug1 in best_augs.keys():
  for aug2 in best_augs.keys():
    for aug3 in best_augs.keys():
      if aug1 != aug2 and aug1 != aug3 and aug2 != aug3:
        x = {
          aug1: best_augs[aug1],
          aug2: best_augs[aug2],
          aug3: best_augs[aug3]
        }
        acc = get_accuracy(model,x)
        print(aug_names[aug1],
              aug_names[aug2],
              aug_names[aug3],
              '->',
              acc)
        s = aug_names[aug1] + '-' + aug_names[aug2] + '-' + aug_names[aug3]
        accuracies.append({s: acc})

Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
v_flip h_flip -> 0.9479048697621744
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
v_flip v_shift -> 0.9450736126840317
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
v_flip h_shift -> 0.9563986409966025
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
v_flip rotate -> 0.934881087202718
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
v_flip crop -> 0.8227633069082673
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
h_flip v_flip -> 0.9479048697621744
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
h_flip v_shift -> 0.950736126840317
Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.
h_flip h_shift -> 0.9558323895809739
Found 1766 images belong

In [ ]:
accuracies

[{'v_flip-True': 0.9490373725934315},
 {'v_flip-False': 0.950736126840317},
 {'h_flip-True': 0.9535673839184597},
 {'h_flip-False': 0.950736126840317},
 {'v_shift-0.25': 0.9518686296715742},
 {'v_shift-0.5': 0.9513023782559457},
 {'h_shift-0.25': 0.9524348810872028},
 {'h_shift-0.5': 0.9524348810872028},
 {'rotate-60': 0.9416761041902605},
 {'rotate-120': 0.9331823329558324},
 {'rotate-180': 0.9479048697621744},
 {'rotate-240': 0.9371460928652322},
 {'rotate-300': 0.9320498301245753},
 {'crop-0.25': 0.9377123442808607},
 {'crop-0.5': 0.8924122310305775},
 {'crop-0.75': 0.7916194790486977},
 {'v_flip-h_flip': 0.9479048697621744},
 {'v_flip-v_shift': 0.9450736126840317},
 {'v_flip-h_shift': 0.9563986409966025},
 {'v_flip-rotate': 0.934881087202718},
 {'v_flip-crop': 0.8227633069082673},
 {'h_flip-v_flip': 0.9479048697621744},
 {'h_flip-v_shift': 0.950736126840317},
 {'h_flip-h_shift': 0.9558323895809739},
 {'h_flip-rotate': 0.9394110985277463},
 {'h_flip-crop': 0.7927519818799547},
 {'v_

In [ ]:
def get_keys(acc, accuracies=accuracies):
  keys=[]
  for el in accuracies:
    val = list(el.values())[0]
    if acc == val:
      keys.append(list(el.keys())[0])
  return keys

In [ ]:
base_accuracy = get_accuracy(model)

Found 1766 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.


In [ ]:
n_top = 10

sorted_accs = list(set([list(x.values())[0] for x in accuracies]))
sorted_accs.sort(reverse=True)

print('base accuracy:', base_accuracy)

for i in range(n_top):
  acc = sorted_accs[i]
  keys = get_keys(acc)
  for key in keys:
    print(i+1,key,acc)

base accuracy: 0.950736126840317
1 v_flip-h_shift 0.9563986409966025
1 h_shift-v_flip 0.9563986409966025
2 h_flip-h_shift 0.9558323895809739
2 h_shift-h_flip 0.9558323895809739
3 v_flip-h_flip-h_shift 0.9552661381653454
3 v_flip-h_shift-h_flip 0.9552661381653454
3 h_flip-v_flip-h_shift 0.9552661381653454
3 h_flip-h_shift-v_flip 0.9552661381653454
3 h_shift-v_flip-h_flip 0.9552661381653454
3 h_shift-h_flip-v_flip 0.9552661381653454
4 h_flip-True 0.9535673839184597
5 h_shift-0.25 0.9524348810872028
5 h_shift-0.5 0.9524348810872028
6 v_shift-0.25 0.9518686296715742
7 v_shift-0.5 0.9513023782559457
8 v_flip-False 0.950736126840317
8 h_flip-False 0.950736126840317
8 h_flip-v_shift 0.950736126840317
8 v_shift-h_flip 0.950736126840317
9 v_flip-True 0.9490373725934315
10 rotate-180 0.9479048697621744
10 v_flip-h_flip 0.9479048697621744
10 h_flip-v_flip 0.9479048697621744
10 v_flip-v_shift-h_shift 0.9479048697621744
10 v_flip-h_shift-v_shift 0.9479048697621744
10 h_flip-v_shift-h_shift 0.947904

In [ ]:
print(best_vs_val)
print(best_hs_val)
print(best_rot_val)
print(best_cr_val)

0.5
0.5
300
0.75
